In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9169144765290712395
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Derma


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv


from hanspell import spell_checker

from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [9]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [10]:
data_org = pd.read_excel('[0501]derma_stemmed.xlsx',index_col=0)
print(data_org.shape)
data_org.head()

(31294, 10)


,h,reviews,stars,review_date,do,si,ro,cleaned_reviews,reviews_stem,reviews_pos
0.0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피', '가려움증', '원한', '병원', '진료', '두피', '현미경', ...","['두피/Noun', '가려움증/Noun', '으로/Josa', '내/Determi..."
1.0,고운나라피부과의원,내과,10,"12월 20일, 2021년",경남,김해시,내외중앙로,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"['대상포진', '진료', '주', '이상', '약', '주사', '진통', '대상...",['내과/Noun']
2.0,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름', '고민', '방문', '피부', '전문의', '증상', '이유', '...","['여드름/Noun', '고민/Noun', '이/Josa', '있어서/Adjecti..."
3.0,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문신', '제거', '문신', '곳일', '부분', '여러', '군데', '워낙...","['문/Modifier', '신/Modifier', '제거/Noun', '문신/No..."
4.0,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장', '설명', '병원', '김해', '곳', '유가', '직원', '전부'...","['원장/Noun', '님/Suffix', '이/Josa', '설명/Noun', '..."


In [11]:
data = data_org.copy()
data = data[['h', 'cleaned_reviews', 'reviews_pos']]
data['work'] = data['h']+'_'+data['reviews_pos']
data.head()

,h,cleaned_reviews,reviews_pos,work
0.0,고운나라피부과의원,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피/Noun', '가려움증/Noun', '으로/Josa', '내/Determi...","고운나라피부과의원_['두피/Noun', '가려움증/Noun', '으로/Josa', ..."
1.0,고운나라피부과의원,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,['내과/Noun'],고운나라피부과의원_['내과/Noun']
2.0,고운나라피부과의원,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름/Noun', '고민/Noun', '이/Josa', '있어서/Adjecti...","고운나라피부과의원_['여드름/Noun', '고민/Noun', '이/Josa', '있..."
3.0,고운나라피부과의원,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문/Modifier', '신/Modifier', '제거/Noun', '문신/No...","고운나라피부과의원_['문/Modifier', '신/Modifier', '제거/Nou..."
4.0,고운나라피부과의원,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장/Noun', '님/Suffix', '이/Josa', '설명/Noun', '...","고운나라피부과의원_['원장/Noun', '님/Suffix', '이/Josa', '설..."


In [12]:
k = data.cleaned_reviews.tolist()[0]
k = okt.pos(k, join = True)
print(k)

['두피/Noun', '가려움증/Noun', '으로/Josa', '내원한/Verb', '병원/Noun', '인데/Josa', '진료/Noun', '끝나고/Verb', '두피/Noun', '현미경/Noun', '사진/Noun', '을/Josa', '무료/Noun', '로/Josa', '찍어서/Verb', '상담/Noun', '해주셨어요/Verb', '제/Noun', '두피/Noun', '에/Josa', '맞는/Verb', '샴푸/Noun', '권하셔서/Verb', '하나/Noun', '사가지고/Verb', '왔어요/Verb', '샴푸/Noun', '방법/Noun', '도/Josa', '자세히/Adjective', '알려주시고/Verb', '참/Verb', '친절하셨어요/Adjective']


In [13]:
data_work = data['work'].tolist()
f=0
morphs_list = []

for i in data_work[:1]:
    hospital = i.split('_')[0]
    sentence = i.split('_')[1]
    sentence = sentence.replace("'",'')
    sentence = sentence.replace(" '",'')
    sent = sentence.split('/Verb')
    # print(sent)
    f+=1 
    for k in sent:
        k = k.replace('/Noun','')
        k = k.replace('/Josa','')
        k = k.replace('/Determiner','')
        k = k.replace('/Adjective','')
        k = k.replace('/Punctuation','')
        k = k.replace('/Modifier','')
        k = k.replace('/Foreign','')
        k = clean_text(k)
        k = k.replace('  ','')
        # k = k.replace(']','')
        # k = k.replace(',','')
        # k = k.replace(' ','')

        k = spell_checker.check(k)
        k = k.checked
        k = k.strip()

        print(hospital,'-',f,'-', k)
    #     pos = okt.pos(k, join = True)
    #     print(pos)
    #     for j in pos:
    #         # print(j)
    #         if j.split('/')[1] == 'Noun':
    #             morphs_list.append(j)
    #         elif j.split('/')[1] == 'Verb':
    #             morphs_list.append(j)
    #         elif j.split('/')[1] == 'Adjective':
    #             morphs_list.append(j)
            
            ###
            # morphs_list.set

고운나라피부과의원 - 1 - 두피 가려움증으로 내 원한 병원인데 진료 끝나고
고운나라피부과의원 - 1 - 두피 현미경 사진을 무료로 찍어서
고운나라피부과의원 - 1 - 상담해주셨어요
고운나라피부과의원 - 1 - 제 두피에 맞는
고운나라피부과의원 - 1 - 샴푸 권하셔서
고운나라피부과의원 - 1 - 하나 사가지고
고운나라피부과의원 - 1 - 왔어요
고운나라피부과의원 - 1 - 샴푸 방법 도 자세히 알려주 시
고운나라피부과의원 - 1 - 고참 친절하셨어요


In [100]:
for i in morphs_list:
    
    print(i)
    print(type(i))

두피 가려움증 으로 내 원한 병원 인데 진료 끝나고
<class 'str'>
 두피 현미경 사진 을 무료 로 찍어서
<class 'str'>
 상담 해주셨어요
<class 'str'>
 . 제 두피 에 맞는
<class 'str'>
 샴푸 권하셔서
<class 'str'>
 하나 사가지고
<class 'str'>
 왔어요
<class 'str'>
 ㆍ 샴푸 방법 도 자세히 알려주시
<class 'str'>
 고참 친절하셨어요 
<class 'str'>


In [15]:
k = 0

for i in data_org.reviews:
    if '내원' in i:
        k+=1
print(k)

1060


In [77]:
data.cleaned_reviews.tolist()[0]

'두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해주셨어요 제 두피에 맞는 샴푸 권하셔서 하나 사가지고 왔어요 샴푸 방법도 자세히 알려주시고 참 친절하셨어요'

In [73]:
data_work[0]

"고운나라피부과의원_['두피/Noun', '가려움증/Noun', '으로/Josa', '내/Determiner', '원한/Noun', '병원/Noun', '인데/Josa', '진료/Noun', '끝나고/Verb', '두피/Noun', '현미경/Noun', '사진/Noun', '을/Josa', '무료/Noun', '로/Josa', '찍어서/Verb', '상담/Noun', '해주셨어요/Verb', './Punctuation', '제/Modifier', '두피/Noun', '에/Josa', '맞는/Verb', '샴푸/Noun', '권하셔서/Verb', '하나/Noun', '사가지고/Verb', '왔어요/Verb', 'ㆍ/Foreign', '샴푸/Noun', '방법/Noun', '도/Josa', '자세히/Adjective', '알려주시/Verb', '고참/Noun', '친절하셨어요/Adjective', ',/Punctuation']"

In [52]:
data_work[0]

"고운나라피부과의원_['두피/Noun', '가려움증/Noun', '으로/Josa', '내/Determiner', '원한/Noun', '병원/Noun', '인데/Josa', '진료/Noun', '끝나고/Verb', '두피/Noun', '현미경/Noun', '사진/Noun', '을/Josa', '무료/Noun', '로/Josa', '찍어서/Verb', '상담/Noun', '해주셨어요/Verb', './Punctuation', '제/Modifier', '두피/Noun', '에/Josa', '맞는/Verb', '샴푸/Noun', '권하셔서/Verb', '하나/Noun', '사가지고/Verb', '왔어요/Verb', 'ㆍ/Foreign', '샴푸/Noun', '방법/Noun', '도/Josa', '자세히/Adjective', '알려주시/Verb', '고참/Noun', '친절하셨어요/Adjective', ',/Punctuation']"

In [18]:
okt.pos('슈링크')

[('슈', 'Noun'), ('링크', 'Noun')]